Ray-tracing des rayons venant d'une lampe dans un chambre de culture

Importation des différents Packages et du script "définition" contenant tous les fonctions utilisées. 

In [ ]:
import json
import sys
import vtk
import numpy
from definition import *
from datetime import datetime

Fonctions qui permettent de convertir list en numpy et vice-versa. 

In [ ]:
l2n = lambda l: numpy.array(l)
n2l = lambda n: list(n)

Création du dictionnaire contenant les résultats
Début du chrono de l'exécution

In [ ]:
plant_result = {}
startTime = datetime.now()

lecture fichier d'entrée .json contenant les données des lampes et de la chambre de culture

In [ ]:
try:
   with open('donnee_entree.json') as f:
     data = json.load(f)
except:
   sys.exit("Error: enable to read file")

Création de l'environnement 3D virtuel avec Renderer et fond blanc

In [ ]:
ren = vtk.vtkRenderer()
ren.SetBackground(1, 1, 1) #(R,G,B)

Création de la chambre de culture en Cube 

In [ ]:
xwidth = data["room"]["width"]
ywidth = data["room"]["height"]
zwidth = data["room"]["ceiling-height"]
cube = vtk.vtkCubeSource()
cube.SetXLength(xwidth)
cube.SetYLength(ywidth)
cube.SetZLength(zwidth)
cube.Update()

Transformation de la géométrie en primitives graphiques

In [ ]:
cubeMapper = vtk.vtkPolyDataMapper()
cubeMapper.SetInputConnection(cube.GetOutputPort())

L'acteur représente l'entitée géométrique.
Il permet de définir sa position, son orientation, sa couleur, etc.

In [ ]:
cubeActor = vtk.vtkActor()
cubeActor.SetMapper(cubeMapper)
cubeActor.SetOrigin(0, 0, 0) #?????
cubeActor.GetProperty().SetOpacity(0.1)
cubeActor.GetProperty().SetColor(0, 0, 1)
cubeActor.GetProperty().EdgeVisibilityOn()  # show edges/wireframe
cubeActor.GetProperty().SetEdgeColor(0, 1, 0)

Ajout de l'acteur à l'environnement virtuel

In [ ]:
ren.AddActor(cubeActor)

In [ ]:
# indiquer que ce sera des surfaces interscetions 
obbSurface = vtk.vtkOBBTree()
obbSurface.SetDataSet(cube.GetOutput())
obbSurface.BuildLocator()

In [ ]:
# Création de 'vtkPolyDataNormals' et le connecter au cube
normalsCalcSurface = vtk.vtkPolyDataNormals()
normalsCalcSurface.SetInputConnection(cube.GetOutputPort())

# Disable normal calculation at cell vertices
normalsCalcSurface.ComputePointNormalsOff()
# Enable normal calculation at cell centers
normalsCalcSurface.ComputeCellNormalsOn()
# Disable splitting of sharp edges
normalsCalcSurface.SplittingOff()
# Disable global flipping of normal orientation
normalsCalcSurface.FlipNormalsOff()
# Enable automatic determination of correct normal orientation
normalsCalcSurface.AutoOrientNormalsOn()
# Perform calculation
normalsCalcSurface.Update()

In [ ]:
# plante
plant = vtk.vtkPlaneSource()
plant.SetOrigin(-30, -75, -10)  #premier point
plant.SetPoint1(-30, -75, 30)   #deuxième point
plant.SetPoint2(40, -75, -10)   #troisième point
plant.SetXResolution(15)
plant.SetYResolution(15)
plant.Update()

plantMapper = vtk.vtkPolyDataMapper()
plantMapper.SetInputConnection(plant.GetOutputPort())

plantActor = vtk.vtkActor()
plantActor.SetMapper(plantMapper)
plantActor.GetProperty().SetColor(0, 1, 0)
plantActor.GetProperty().EdgeVisibilityOn()  # show edges/wireframe
plantActor.GetProperty().SetEdgeColor(0, 1, 0)

ren.AddActor(plantActor)

In [ ]:
# indiquer que ce sera des surfaces interscetions 
obbPlant = vtk.vtkOBBTree()
obbPlant.SetDataSet(plant.GetOutput())
obbPlant.BuildLocator()

In [ ]:
v=0

x = data["array"][str(v)]["x"] #par rapport à l'origine qui est au centre du cube
y = data["array"][str(v)]["y"]
z = data["array"][str(v)]["z"]

Lamp = vtk.vtkSphereSource()
Lamp.SetCenter(x, y, z)
Lamp.SetRadius(5)
Lamp.SetStartTheta(data["array"][str(v)]["angle"])
Lamp.SetPhiResolution(30)
Lamp.SetThetaResolution(30)

mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(Lamp.GetOutputPort())

pointActor = vtk.vtkActor()
pointActor.SetMapper(mapper)
pointActor.SetOrigin(0, 0, 0) 
pointActor.GetProperty().SetColor(color(data["array"][str(v)]["wavelength"]))
pointActor.GetProperty().EdgeVisibilityOn()  # show edges/wireframe
pointActor.GetProperty().SetEdgeColor(color(data["array"][str(v)]["wavelength"]))

ren.AddActor(pointActor)

In [ ]:
cellCenterCalcLamp = vtk.vtkCellCenters()
cellCenterCalcLamp.SetInputConnection(Lamp.GetOutputPort())
cellCenterCalcLamp.Update()
    
#on extrait le point numéro 0 de la mesh qui correspond au centre du mesh.
pointsCellCentersLamp = cellCenterCalcLamp.GetOutput(0)

In [ ]:
# Création de 'vtkPolyDataNormals()' et le connecter à la lampe
normalsCalcLamp = vtk.vtkPolyDataNormals()
normalsCalcLamp.SetInputConnection(Lamp.GetOutputPort())

# Disable normal calculation at cell vertices
normalsCalcLamp.ComputePointNormalsOff()
# Enable normal calculation at cell centers
normalsCalcLamp.ComputeCellNormalsOn()
# Disable splitting of sharp edges
normalsCalcLamp.SplittingOff()
# Disable global flipping of normal orientation
normalsCalcLamp.FlipNormalsOff()
# Enable automatic determination of correct normal orientation
normalsCalcLamp.AutoOrientNormalsOn()
# Perform calculation
normalsCalcLamp.Update()

In [ ]:
#Extract the normal-vector data at the sun's cells
normalsLamp = normalsCalcLamp.GetOutput().GetCellData().GetNormals()
# Extract the normal-vector data at the earth's cells
normalsSurface = normalsCalcSurface.GetOutput().GetCellData().GetNormals()
 

In [ ]:
#Pour chaque point central de surface mesh de la lampe
i=0
for idx in range(pointsCellCentersLamp.GetNumberOfPoints()):
# Get coordinates of lamp's cell center
   pointLamp = pointsCellCentersLamp.GetPoint(idx)
 # Get normal vector at that cell
   normalLamp = normalsLamp.GetTuple(idx)
    
# Calculate the 'target' of the ray 
   pointRayTarget = n2l(l2n(pointLamp) + 500*l2n(normalLamp))

# Check if there are any intersections for the given ray with surface
   if isHit(obbSurface, pointLamp, pointRayTarget):  # intersections were found
        # Retrieve coordinates of intersection points and intersected cell ids
        pointsInter, cellIdsInter = GetIntersect(obbSurface, pointLamp, pointRayTarget)
        # Render lines/rays emanating from the slamp. 
        addLine(ren, pointLamp, pointsInter[0], color(data["array"][str(v)]["wavelength"]), opacity = 0.1)
        # Render intersection points
        point = addPoint(ren, pointsInter[0], color(data["array"][str(v)]["wavelength"]), opacity = 0.1)
        # Calculate the incident ray vector
        # Get the normal vector at the surface cell that intersected with the ray
        normalSurface = normalsSurface.GetTuple(cellIdsInter[0])
        vecInc = n2l(l2n(pointRayTarget) - l2n(pointLamp))
        # Calculate the reflected ray vector
        vecRef = calcVecR(vecInc, normalSurface)
        # Calculate the 'target' of the reflected ray 
        pointRayReflectedTarget = n2l(l2n(pointsInter[0]) + l2n(vecRef))
        
        # Fresnel 
        R, T =Fresnel(data["room"]["n_air"],data["room"]["n_paroi"],vecInc,normalSurface)
    # Check if there are any intersections for the given ray with plante  
    
   if isHit(obbPlant, pointLamp, pointRayTarget):  # intersections were found
        # Retrieve coordinates of intersection points and intersected cell ids
        ptsInter, cIdsInter = GetIntersect_plant(obbPlant, pointLamp, pointRayTarget)
        # Render lines/rays emanating from the slamp. 
        addLine(ren, pointLamp, ptsInter[0], color(data["array"][str(v)]["wavelength"]))
        # Render intersection points
        addPoint(ren, ptsInter[0], color(data["array"][str(v)]["wavelength"]))
        
        key = "surface_"+str(cIdsInter[0])
          
        if key in plant_result: 
              plant_result["surface_"+str(cIdsInter[0])]['coord'][i] = ptsInter[0]
              plant_result["surface_"+str(cIdsInter[0])]['lampe'][i] = v
              plant_result["surface_"+str(cIdsInter[0])]['Rayonnement'][i] = 1
              i+=1
              
             
        else:
             plant_result.update({"surface_"+str(cIdsInter[0]):{"coord":{},"Rayonnement":{},"lampe":{}}})
             plant_result["surface_"+str(cIdsInter[0])]['coord'][i]= ptsInter[0]
             plant_result["surface_"+str(cIdsInter[0])]['lampe'][i] = v
             plant_result["surface_"+str(cIdsInter[0])]['Rayonnement'][i] = 1
             i+=1
                # Check if there are any intersections for the given ray with plante  
   if isHit(obbPlant, pointLamp, pointRayTarget):  # intersections were found
        # Retrieve coordinates of intersection points and intersected cell ids
        ptsInter, cIdsInter = GetIntersect_plant(obbPlant, pointLamp, pointRayTarget)
        # Render lines/rays emanating from the slamp. 
        addLine(ren, pointLamp, ptsInter[0], color(data["array"][str(v)]["wavelength"]))
        # Render intersection points
        addPoint(ren, ptsInter[0], color(data["array"][str(v)]["wavelength"]))
        
        key = "surface_"+str(cIdsInter[0])
          
        if key in plant_result: 
              plant_result["surface_"+str(cIdsInter[0])]['coord'][i] = ptsInter[0]
              plant_result["surface_"+str(cIdsInter[0])]['lampe'][i] = v
              plant_result["surface_"+str(cIdsInter[0])]['Rayonnement'][i] = 1
              i+=1
              
             
        else:
             plant_result.update({"surface_"+str(cIdsInter[0]):{"coord":{},"Rayonnement":{},"lampe":{}}})
             plant_result["surface_"+str(cIdsInter[0])]['coord'][i]= ptsInter[0]
             plant_result["surface_"+str(cIdsInter[0])]['lampe'][i] = v
             plant_result["surface_"+str(cIdsInter[0])]['Rayonnement'][i] = 1
             i+=1
   if data["room"]["reflection"] == "speculaire":
      # Check if there are any intersections for the given ray
          # Check if there are any intersections for the given ray with plante  
       if isHit(obbSurface,  pointsInter[0], pointRayReflectedTarget):  # intersections were found
          # Retrieve coordinates of intersection points and intersected cell ids
            pointsInter, cellIdsInter = GetIntersect(obbSurface,  pointsInter[0], pointRayReflectedTarget)
     
            addLine(ren,  pointsInter[0], pointsInter[1], color(data["array"][str(v)]["wavelength"]),opacity=0.1)
            addPoint(ren, pointsInter[1], color(data["array"][str(v)]["wavelength"]), opacity =0.1)
         
           # Calculate the incident ray vector
           # Get the normal vector at the earth cell that intersected with the ray
            normalSurface = normalsSurface.GetTuple(cellIdsInter[1])
            vecInc = n2l(l2n(pointRayReflectedTarget) - l2n(pointsInter[1]))
           # Calculate the reflected ray vector
            vecRef = calcVecR(vecInc, normalSurface)
            
            pointRayReflectedTarget = n2l(l2n(pointsInter[1]) + l2n(vecRef))
     # Check if there are any intersections for the given ray with plante  
       if isHit(obbPlant, pointsInter[0], pointRayReflectedTarget):  # intersections were found
        # Retrieve coordinates of intersection points and intersected cell ids
           ptsInter, cIdsInter = GetIntersect_plant(obbPlant,  pointsInter[0], pointRayReflectedTarget)
        # Render lines/rays emanating from the slamp. 
           addLine(ren, pointsInter[0], ptsInter[0], color(data["array"][str(v)]["wavelength"]))
        # Render intersection points
           addPoint(ren, ptsInter[0], color(data["array"][str(v)]["wavelength"]))
        
           key = "surface_"+str(cIdsInter[0])
        
           if key in plant_result: 
              plant_result["surface_"+str(cIdsInter[0])]['coord'][i] = ptsInter[0]
              plant_result["surface_"+str(cIdsInter[0])]['lampe'][i] = v
              plant_result["surface_"+str(cIdsInter[0])]['Rayonnement'][i] = R
              i+=1
            
             
           else:
             plant_result.update({"surface_"+str(cIdsInter[0]):{"coord":{},"Rayonnement":{},"lampe":{}}})
             plant_result["surface_"+str(cIdsInter[0])]['coord'][i]= ptsInter[0]
             plant_result["surface_"+str(cIdsInter[0])]['lampe'][i] = v
             plant_result["surface_"+str(cIdsInter[0])]['Rayonnement'][i] = R

In [ ]:
# nombre de rayons atteint la plante
print(str(i-1)+" rayons ont atteint la plante" )
print("Chaque lampe a émis "+str(idx)+" rayons")

In [ ]:
# durée de l'execution du programme
endTime = datetime.now()
deltaTime = endTime - startTime
print("Completed in "+str(deltaTime))

In [ ]:
with open("out_1_lamp.json", "w") as f:
        json.dump(plant_result, f,indent= 8)

In [ ]:
#renWin = vtk.vtkRenderWindow()
#renWin.AddRenderer(ren)

#iren = vtk.vtkRenderWindowInteractor()
#iren.SetRenderWindow(renWin)

#iren.Initialize()
#renWin.Render()
#iren.Start()